# Check why Metrolink (ITP ID: 323) is missing from routes shapefile

In [1]:
import geopandas as gpd
import pandas as pd

from calitp.tables import tbl
from calitp import query_sql
from siuba import *

from shared_utils import utils
import prep_data

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [ ]:
#utils.download_geoparquet(prep_data.GCS_FILE_PATH, 
#                          "ca_transit_routes", save_locally=True)

ITP ID 323 is missing in the `ca_transit_routes.parquet`.

## Initial queries
* Check that 323 shows up there -- it does.

In [4]:
SELECTED_DATE = "2022-06-21"
#prep_data.create_local_parquets(prep_data.SELECTED_DATE) 

In [3]:
files = ["agencies", "latest_itp_id", "route_info", 
         "stops", "trips", "routes"
        ]

check_itp_id = 323

def check_file_for_itp_id(file, itp_id):
    filename = f"./data/{file}.parquet"
    
    if file=="routes":
        df = gpd.read_parquet(filename)
    else:
        df = pd.read_parquet(filename)
    
    subset = df[df.calitp_itp_id==itp_id]
    display(f"File: {file}")
    display(subset.head())
    
for f in files:
    check_file_for_itp_id(f, check_itp_id)

'File: agencies'

,calitp_itp_id,agency_id,agency_name
156,323,Metrolink,Metrolink Trains


'File: latest_itp_id'

,calitp_itp_id
77,323


'File: route_info'

...

,calitp_itp_id,stop_id,stop_lat,stop_lon,stop_name,stop_code
51051,323,109,34.093639,-117.753052,Pomona (North) Metrolink Station,None
51052,323,113,34.211559,-118.448219,Van Nuys Metrolink Station,None
51053,323,165,34.497860,-118.118301,Vincent Grade/Acton Metrolink Station,None
51054,323,103,34.253197,-118.599411,Chatsworth Metrolink Station,None
51055,323,101,34.086426,-117.957397,Baldwin Park Metrolink Station,None


'File: trips'

,calitp_itp_id,route_id,shape_id
21,323,San Bernardino Line,None
618,323,Inland Emp.-Orange Co. Line,None
1585,323,Riverside Line,None
1755,323,Antelope Valley Line,None
3149,323,Ventura County Line,None


'File: routes'

,calitp_itp_id,shape_id,geometry
11065,323,91in,"LINESTRING (-117.20562 33.76391, -117.20830 33..."
11066,323,91out,"LINESTRING (-118.23377 34.05600, -118.23343 34..."
11067,323,AVin,"LINESTRING (-118.13613 34.69658, -118.11499 34..."
11068,323,AVout,"LINESTRING (-118.23482 34.05420, -118.23229 34..."
11069,323,IEOCin,"LINESTRING (-117.37573 33.18759, -117.38783 33..."


## Trips query

Go look at original query for `trips` without getting rid of any columns. Just in case.

In [10]:
# Trips query
dim_trips = (tbl.views.gtfs_schedule_dim_trips()
            #>> filter(_.calitp_itp_id != 200, _.calitp_itp_id != 0)
             #>> select(*trip_cols, _.trip_key)
             >> filter(_.calitp_itp_id==check_itp_id)
             >> distinct()
            )

dim_trips

,calitp_itp_id,calitp_url_number,route_id,service_id,trip_id,shape_id,trip_headsign,trip_short_name,direction_id,block_id,wheelchair_accessible,bikes_allowed,calitp_extracted_at,calitp_hash,trip_key,calitp_deleted_at
0,323,0,Orange County Line,1152,296800433,None,Oceanside,668,0,None,None,None,2021-10-28,KJ11lcHqmnqB3TJNnE4YPw==,4426110514318030117,2099-01-01
1,323,0,Antelope Valley Line,1142,200090225,None,Lancaster,225,0,None,None,None,2022-04-05,l3cqZlOUX9SjGAI/zer8QQ==,-1412446666788434424,2099-01-01
2,323,0,91 Line,1150,200090709,None,L.A. Union Station,709,1,None,None,None,2022-04-05,DJArUXzOYpRArfGDLozZSg==,-154331551297351104,2099-01-01
3,323,0,San Bernardino Line,1146,200090308,None,San Bernardino - Downtown,308,0,None,None,None,2022-04-05,xxLjiiA5uuNrNTSVvo1eYg==,-5084664679889279126,2099-01-01
4,323,0,Ventura County Line,1141,294200148,None,L.A. Union Station,148,1,None,None,None,2022-04-05,yt5KJTODnuJDQs9ZOdu+0g==,4873550634793634261,2099-01-01


In [11]:
(dim_trips >> collect()).shape_id.value_counts()

Series([], Name: shape_id, dtype: int64)

In [13]:
trips = (tbl.views.gtfs_schedule_fact_daily_trips()
         >> filter(_.service_date == SELECTED_DATE, 
                   _.is_in_service==True)
         >> select(_.trip_key, _.service_date)
         >> inner_join(_, dim_trips, on = "trip_key")
         #>> select(*trip_cols)
         >> distinct()
         >> collect()
        )

In [15]:
trips.shape_id.value_counts()

Series([], Name: shape_id, dtype: int64)